#  WeRateDogs Project

# Introduction 

In this project we are going to do data wrangling process. Data Wrangling is one of the most important process in data analysis. Data Warngling flow works like *Gather,Access* and *Clean*,  once it is done we get down to solving problem by analysis, moreover data wrangling is an iterative process we can get back the wrangling process even after we start analysing our data's through plots and graphs.

Through out this project we will go in a step by step process of **Data Wrangling**,
- Gather
- Access 
- Clean 

# Gather 

In this Project we are going to use [WeRateDogs](https://en.wikipedia.org/wiki/WeRateDogs) for the data Wrangling Process.

- The WeRateDogs Twitter archive. We will download this file manually by clicking the following link: twitter_archive_enhanced.csv
- The tweet image predictions, i.e., what breed of dog (or other object, animal, etc.) is present in each tweet according to a neural network. This file (image_predictions.tsv) hosted on Udacity's servers and should be downloaded programmatically using the Requests library and the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv
- Each tweet's retweet count and favorite (i.e. "like") count at minimum, and any additional data we will find interesting. Using the tweet IDs in the WeRateDogs Twitter archive, we will query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called tweet_json.txt file. Each tweet's JSON data should be written to its own line. Then we will read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count.

In [1]:
! pip install --user tweepy

In [2]:
# import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tweepy
import json
import time
import requests
import re

In [3]:
# read the twitter archived file
df = pd.read_csv('twitter-archive-enhanced.csv')

In [4]:
# display data in the twitter archived file
df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [5]:
# check number of rows & columns using the info function
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

It has <b> 2356 rows </b> & <b> 17 columns </b>

In [6]:
# import image-prediction.tsv file
image_prediction = pd.read_csv("image-predictions.tsv", delimiter='\t', encoding='utf-8')

In [7]:
#display data from image_prediction
image_prediction.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [8]:
# check number of rows & columns
image_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


It has <b> 2075 rows </b> & <b> 12 columns </b>

In [9]:
# twitter api

consumer_key = 'XXXX'
consumer_secret = 'XXXX'
access_token = 'XXXX'
access_secret = 'XXXX'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

# API instance
api = tweepy.API(auth,
                parser = tweepy.parsers.JSONParser(), # parse the result to Json Object
                wait_on_rate_limit = True, # automatically wait for rate limits to replenish
                wait_on_rate_limit_notify = True) # print a notification when Tweepy is waiting for rate limits to replenish

In [10]:
# list where we store dictionaries of result
df_list = []
# list where we store the errors
errors_list = []

# calculate start time for execution
start_timing = time.time()

# get all the tweets in the tweets in archive dataframe
for tweet_id in df['tweet_id']:
    try:
        tweet_data = api.get_status(tweet_id, tweet_mode = 'extended')
        # to get names of attributes
        # print(json.dumps(tweet_data, indent = 4))
        # break
        
        favorite_count = tweet_data['favorite_count'] # count favorites the tweet had
        retweet_count = tweet_data['retweet_count'] # count of the retweet
        user_followers_count = tweet_data['user']['followers_count'] # count followers the user had
        user_favourites_count = tweet_data['user']['favourites_count'] # count favorites the user had
        date_time = tweet_data['created_at']
        
        df_list.append({'tweet_id': int(tweet_id),
                        'favorites': int(favorite_count),
                        'retweets': int(retweet_count),
                        'user_followers': int(user_followers_count),
                        'user_favourites': int(user_favourites_count),
                        'date_time': pd.to_datetime(date_time)})
        
    # catch exceptions
    except Exception as e:
        print(str(tweet_id)+ " - " + str(e))
        errors_list.append(tweet_id)
        
# time of execution
end_timing = time.time()
total_time = end_timing - start_timing
print('Execution time:', total_time)

888202515573088257 - [{'code': 144, 'message': 'No status found with that ID.'}]
873697596434513921 - [{'code': 144, 'message': 'No status found with that ID.'}]
872668790621863937 - [{'code': 144, 'message': 'No status found with that ID.'}]
872261713294495745 - [{'code': 144, 'message': 'No status found with that ID.'}]
869988702071779329 - [{'code': 144, 'message': 'No status found with that ID.'}]
866816280283807744 - [{'code': 144, 'message': 'No status found with that ID.'}]
861769973181624320 - [{'code': 144, 'message': 'No status found with that ID.'}]
856602993587888130 - [{'code': 144, 'message': 'No status found with that ID.'}]
851953902622658560 - [{'code': 144, 'message': 'No status found with that ID.'}]
845459076796616705 - [{'code': 144, 'message': 'No status found with that ID.'}]
844704788403113984 - [{'code': 144, 'message': 'No status found with that ID.'}]
842892208864923648 - [{'code': 144, 'message': 'No status found with that ID.'}]
837366284874571778 - [{'code

Rate limit reached. Sleeping for: 740


754011816964026368 - [{'code': 144, 'message': 'No status found with that ID.'}]
680055455951884288 - [{'code': 144, 'message': 'No status found with that ID.'}]


Rate limit reached. Sleeping for: 744


676975532580409345 - Failed to send request: ('Connection aborted.', OSError("(54, 'ECONNRESET')"))
Execution time: 4537.796627044678


In [11]:
# length of result 
len(df_list)

2331

In [12]:
# lenght of errors
len(errors_list)

25

In [13]:
# perform same operation for errors_list
e_list = []
for e in errors_list:
    try:
        favorite_count = tweet_data['favorite_count'] # count favorites the tweet had
        retweet_count = tweet_data['retweet_count'] # count of the retweet
        user_followers_count = tweet_data['user']['followers_count'] # count followers the user had
        user_favourites_count = tweet_data['user']['favourites_count'] # count favorites the user had
        date_time = tweet_data['created_at']
        
        df_list.append({'tweet_id': int(tweet_id),
                        'favorites': int(favorite_count),
                        'retweets': int(retweet_count),
                        'user_followers': int(user_followers_count),
                        'user_favourites': int(user_favourites_count),
                        'date_time': pd.to_datetime(date_time)})
        
    except Exception:
        print(str(tweet_id)+ " _ " + str(e))
        e_list.append(e)

In [14]:
len(df_list)

2356

Lenght of result after querying result is <b> 2356 </b>

In [15]:
# create dataframe for the result
df_final_tweets = pd.DataFrame(df_list, columns=['tweet_id', 'favorites', 'retweets', 'user_followers', 
                                                 'user_favourites', 'date_time'])

In [16]:
# data in df_final_tweets
df_final_tweets.head()

,tweet_id,favorites,retweets,user_followers,user_favourites,date_time
0,892420643555336193,37200,7969,8310259,142725,2017-08-01 16:23:56+00:00
1,892177421306343426,31996,5913,8310259,142725,2017-08-01 00:17:27+00:00
2,891815181378084864,24092,3909,8310259,142725,2017-07-31 00:18:03+00:00
3,891689557279858688,40514,8118,8310259,142725,2017-07-30 15:58:51+00:00
4,891327558926688256,38742,8801,8310259,142725,2017-07-29 16:00:24+00:00


In [17]:
# save the dataframe to a file
df_final_tweets.to_csv('final_tweets.csv', encoding='utf-8', index=False)

In [18]:
df_final_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 6 columns):
tweet_id           2356 non-null int64
favorites          2356 non-null int64
retweets           2356 non-null int64
user_followers     2356 non-null int64
user_favourites    2356 non-null int64
date_time          2356 non-null datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(5)
memory usage: 110.5 KB


There are no missing values

<h2>Assess</h2>

In [19]:
df

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [21]:
df['rating_numerator'].value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [22]:
df['rating_denominator'].value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [23]:
image_prediction

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


In [24]:
image_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [25]:
image_prediction['jpg_url'].value_counts()

https://pbs.twimg.com/media/CvyVxQRWEAAdSZS.jpg                                            2
https://pbs.twimg.com/media/CsGnz64WYAEIDHJ.jpg                                            2
https://pbs.twimg.com/media/ChK1tdBWwAQ1flD.jpg                                            2
https://pbs.twimg.com/ext_tw_video_thumb/807106774843039744/pu/img/8XZg1xW35Xp2J6JW.jpg    2
https://pbs.twimg.com/media/CtVAvX-WIAAcGTf.jpg                                            2
https://pbs.twimg.com/media/CzG425nWgAAnP7P.jpg                                            2
https://pbs.twimg.com/media/CxqsX-8XUAAEvjD.jpg                                            2
https://pbs.twimg.com/media/CW88XN4WsAAlo8r.jpg                                            2
https://pbs.twimg.com/media/CpmyNumW8AAAJGj.jpg                                            2
https://pbs.twimg.com/media/CmoPdmHW8AAi8BI.jpg                                            2
https://pbs.twimg.com/media/CkjMx99UoAM2B1a.jpg                       

In [26]:
image_prediction[image_prediction['jpg_url'] == 'https://pbs.twimg.com/media/C12whDoVEAALRxa.jpg']

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1715,819004803107983360,https://pbs.twimg.com/media/C12whDoVEAALRxa.jpg,1,standard_poodle,0.351308,True,toy_poodle,0.271929,True,Tibetan_terrier,0.094759,True
1718,819015337530290176,https://pbs.twimg.com/media/C12whDoVEAALRxa.jpg,1,standard_poodle,0.351308,True,toy_poodle,0.271929,True,Tibetan_terrier,0.094759,True


In [27]:
df_final_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 6 columns):
tweet_id           2356 non-null int64
favorites          2356 non-null int64
retweets           2356 non-null int64
user_followers     2356 non-null int64
user_favourites    2356 non-null int64
date_time          2356 non-null datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(5)
memory usage: 110.5 KB


<h2> Quality </h2>

<h3> df Dataset </h3>
<li> Several columns have NaN value </li>
<li> in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_id_user_id should be interger </li>
<li> retweeted_status_timestamp, timestamp should be datetime </li>
<li> name column has values like 'a', 'an', 'none' </li>
<li> retweeted_status_timestamp, timestamp should be datetime </li>

<h3>image_prediction dataset</h3>
<li> missing values (2075 rows instead of 2356) </li>
<li> some url values are repeated </li>

<h1> Tidiness </h1>

<li> various dog stages can be in single columns instead of seperate rows </li>
<li> extra information in image_prediction dataset (tweet_id, jpg_url data is sufficient) </li>
<li> all tables can be a part of single dataset </li>

<h1> Cleaning </h1>

In [28]:
# combine all datasets in one dataset
# combine df & image_prediction using merge function
df_master = pd.merge(df, image_prediction, how='left', on=['tweet_id'])

In [29]:
# combine df_master & df_final_tweets
df_master = pd.merge(df_master, df_final_tweets, how='left', on=['tweet_id'])

In [30]:
df_master.to_csv('df_master.csv', encoding='utf-8', index=False)

In [31]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2381 entries, 0 to 2380
Data columns (total 33 columns):
tweet_id                      2381 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2381 non-null object
source                        2381 non-null object
text                          2381 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2322 non-null object
rating_numerator              2381 non-null int64
rating_denominator            2381 non-null int64
name                          2381 non-null object
doggo                         2381 non-null object
floofer                       2381 non-null object
pupper                        2381 non-null object
puppo                         2381 non-null object
jpg_url                       21

In [32]:
df_master.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,p2_conf,p2_dog,p3,p3_conf,p3_dog,favorites,retweets,user_followers,user_favourites,date_time
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,...,0.085851,False,banana,0.076110,False,37200.0,7969.0,8310259.0,142725.0,2017-08-01 16:23:56+00:00
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,...,0.090647,True,papillon,0.068957,True,31996.0,5913.0,8310259.0,142725.0,2017-08-01 00:17:27+00:00
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,...,0.078253,True,kelpie,0.031379,True,24092.0,3909.0,8310259.0,142725.0,2017-07-31 00:18:03+00:00
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,...,0.168086,True,spatula,0.040836,False,40514.0,8118.0,8310259.0,142725.0,2017-07-30 15:58:51+00:00
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,...,0.225770,True,German_short-haired_pointer,0.175219,True,38742.0,8801.0,8310259.0,142725.0,2017-07-29 16:00:24+00:00


In [33]:
# delete retweeted_status_id
df_master = df_master[pd.isnull(df_master.retweeted_status_id)]
# drop duplicates
df_master = df_master.drop_duplicates()
# Delete tweets with no pictures
df_master = df_master.dropna(subset = ['jpg_url'])

In [34]:
len(df_master)

1994

In [35]:
# delete columns we don't need
df_master = df_master.drop('retweeted_status_id', 1)
df_master = df_master.drop('retweeted_status_user_id', 1)
df_master = df_master.drop('retweeted_status_timestamp', 1)
df_master = df_master.drop('date_time', 1)

In [36]:
list(df_master)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog',
 'favorites',
 'retweets',
 'user_followers',
 'user_favourites']

In [37]:
# combine 'doggo', 'floofer', 'pupper', 'puppo' columns using melt function
columns_to_melt = ['doggo', 'floofer', 'pupper', 'puppo']
columns_to_keep = [x for x in df_master.columns.tolist() if x not in columns_to_melt]

# melt the columns
df_master = pd.melt(df_master, id_vars=columns_to_keep, value_vars=columns_to_melt, 
                    var_name='stage', value_name='dog_stages')

In [38]:
df_master = df_master.drop('stage', 1)

In [39]:
# remove prediction columns

predictions = []
confidence = []

def prediction_confidence(df_master):
    if df_master['p1_dog'] == True:
        predictions.append(df_master['p1'])
        confidence.append(df_master['p1_conf'])
    elif df_master['p2_dog'] == True:
        predictions.append(df_master['p2'])
        confidence.append(df_master['p2_conf'])
    elif df_master['p3_dog'] == True:
        predictions.append(df_master['p3'])
        confidence.append(df_master['p3_conf'])
    else:
        predictions.append('NaN')
        confidence.append(0)

df_master.apply(prediction_confidence, axis=1)
df_master['predictions'] = predictions
df_master['confidence'] = confidence

# drop the other prediction columns
df_master = df_master.drop(['img_num', 'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'], 1)

In [40]:
list(df_master)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'jpg_url',
 'favorites',
 'retweets',
 'user_followers',
 'user_favourites',
 'dog_stages',
 'predictions',
 'confidence']

In [41]:
# drop the columns 'in_reply_to_status_id', 'in_reply_to_status_id', 'user_favourites'
df_master = df_master.drop(['in_reply_to_status_id', 'in_reply_to_user_id', 'user_favourites'], 1)

In [42]:
df_master.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,jpg_url,favorites,retweets,user_followers,dog_stages,predictions,confidence
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,37200.0,7969.0,8310259.0,None,NaN,0.000000
1,892177421306343426,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,31996.0,5913.0,8310259.0,None,Chihuahua,0.323581
2,891815181378084864,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,24092.0,3909.0,8310259.0,None,Chihuahua,0.716012
3,891689557279858688,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,40514.0,8118.0,8310259.0,None,Labrador_retriever,0.168086
4,891327558926688256,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,38742.0,8801.0,8310259.0,None,basset,0.555712


In [43]:
# clean content of source
df_master['source'] = df_master['source'].apply(lambda x: re.findall(r'>(.*)<', x)[0])

df_master

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,jpg_url,favorites,retweets,user_followers,dog_stages,predictions,confidence
0,892420643555336193,2017-08-01 16:23:56 +0000,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,37200.0,7969.0,8310259.0,None,NaN,0.000000
1,892177421306343426,2017-08-01 00:17:27 +0000,Twitter for iPhone,This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,31996.0,5913.0,8310259.0,None,Chihuahua,0.323581
2,891815181378084864,2017-07-31 00:18:03 +0000,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,24092.0,3909.0,8310259.0,None,Chihuahua,0.716012
3,891689557279858688,2017-07-30 15:58:51 +0000,Twitter for iPhone,This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,40514.0,8118.0,8310259.0,None,Labrador_retriever,0.168086
4,891327558926688256,2017-07-29 16:00:24 +0000,Twitter for iPhone,This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,38742.0,8801.0,8310259.0,None,basset,0.555712
5,891087950875897856,2017-07-29 00:08:17 +0000,Twitter for iPhone,Here we have a majestic great white breaching ...,https://twitter.com/dog_rates/status/891087950...,13,10,None,https://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,19473.0,2927.0,8310259.0,None,Chesapeake_Bay_retriever,0.425595
6,890971913173991426,2017-07-28 16:27:12 +0000,Twitter for iPhone,Meet Jax. He enjoys ice cream so much he gets ...,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,https://pbs.twimg.com/media/DF1eOmZXUAALUcq.jpg,11363.0,1942.0,8310259.0,None,Appenzeller,0.341703
7,890729181411237888,2017-07-28 00:22:40 +0000,Twitter for iPhone,When you watch your owner call another dog a g...,https://twitter.com/dog_rates/status/890729181...,13,10,None,https://pbs.twimg.com/media/DFyBahAVwAAhUTd.jpg,62756.0,17747.0,8310259.0,None,Pomeranian,0.566142
8,890609185150312448,2017-07-27 16:25:51 +0000,Twitter for iPhone,This is Zoey. She doesn't want to be one of th...,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,https://pbs.twimg.com/media/DFwUU__XcAEpyXI.jpg,26789.0,4029.0,8310259.0,None,Irish_terrier,0.487574
9,890240255349198849,2017-07-26 15:59:51 +0000,Twitter for iPhone,This is Cassie. She is a college pup. Studying...,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,https://pbs.twimg.com/media/DFrEyVuW0AAO3t9.jpg,30691.0,6933.0,8310259.0,doggo,Pembroke,0.511319


In [44]:
#convert column text values to lower case
df_master['text'] = df_master['text'].str.lower()

<h1> Determine Dog Gender </h1>

In [45]:
dog_gender = []

male = ['he', "he's", 'him', 'himself', 'his']
female = ['she', "she's", 'her', 'herself', "her's"]

# loop through the text to check if it contains the male and female pronoun
for text in df_master['text']:
    if any(map(lambda x: x in male, text.split())):
        dog_gender.append('male')
    elif any(map(lambda x: x in female, text.split())):
        dog_gender.append('female')
    else:
        dog_gender.append('NaN')

df_master['gender'] = dog_gender

In [46]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7976 entries, 0 to 7975
Data columns (total 16 columns):
tweet_id              7976 non-null int64
timestamp             7976 non-null object
source                7976 non-null object
text                  7976 non-null object
expanded_urls         7976 non-null object
rating_numerator      7976 non-null int64
rating_denominator    7976 non-null int64
name                  7976 non-null object
jpg_url               7976 non-null object
favorites             7944 non-null float64
retweets              7944 non-null float64
user_followers        7944 non-null float64
dog_stages            7976 non-null object
predictions           7976 non-null object
confidence            7976 non-null float64
gender                7976 non-null object
dtypes: float64(4), int64(3), object(9)
memory usage: 997.1+ KB


In [47]:
# convert all null value to none
df_master.loc[df_master['predictions'] == 'NaN', 'predictions'] = None
df_master.loc[df_master['dog_stages'] == 'NaN', 'dog_stages'] = None

<h1> Convert columns to appropriate field </h1>

In [48]:
df_master.dtypes

tweet_id                int64
timestamp              object
source                 object
text                   object
expanded_urls          object
rating_numerator        int64
rating_denominator      int64
name                   object
jpg_url                object
favorites             float64
retweets              float64
user_followers        float64
dog_stages             object
predictions            object
confidence            float64
gender                 object
dtype: object

In [49]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7976 entries, 0 to 7975
Data columns (total 16 columns):
tweet_id              7976 non-null int64
timestamp             7976 non-null object
source                7976 non-null object
text                  7976 non-null object
expanded_urls         7976 non-null object
rating_numerator      7976 non-null int64
rating_denominator    7976 non-null int64
name                  7976 non-null object
jpg_url               7976 non-null object
favorites             7944 non-null float64
retweets              7944 non-null float64
user_followers        7944 non-null float64
dog_stages            7976 non-null object
predictions           6744 non-null object
confidence            7976 non-null float64
gender                7976 non-null object
dtypes: float64(4), int64(3), object(9)
memory usage: 997.1+ KB


In [50]:
df_master.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,jpg_url,favorites,retweets,user_followers,dog_stages,predictions,confidence,gender
0,892420643555336193,2017-08-01 16:23:56 +0000,Twitter for iPhone,this is phineas. he's a mystical boy. only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,37200.0,7969.0,8310259.0,None,None,0.000000,male
1,892177421306343426,2017-08-01 00:17:27 +0000,Twitter for iPhone,this is tilly. she's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,31996.0,5913.0,8310259.0,None,Chihuahua,0.323581,female
2,891815181378084864,2017-07-31 00:18:03 +0000,Twitter for iPhone,this is archie. he is a rare norwegian pouncin...,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,24092.0,3909.0,8310259.0,None,Chihuahua,0.716012,male
3,891689557279858688,2017-07-30 15:58:51 +0000,Twitter for iPhone,this is darla. she commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,40514.0,8118.0,8310259.0,None,Labrador_retriever,0.168086,female
4,891327558926688256,2017-07-29 16:00:24 +0000,Twitter for iPhone,this is franklin. he would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,38742.0,8801.0,8310259.0,None,basset,0.555712,male


In [51]:
df_master['tweet_id'] = df_master['tweet_id'].astype(object)
df_master['timestamp'] = pd.to_datetime(df_master.timestamp)
df_master['source'] = df_master['source'].astype('category')
#df_master['favorites'] = df_master['favorites'].astype(int)
#df_master['retweets'] = df_master['retweets'].astype(int)
#df_master['user_followers'] = df_master['user_followers'].astype(int)
df_master['dog_stages'] = df_master['dog_stages'].astype('category')
df_master['gender'] = df_master['gender'].astype('category')

<h1> Storing, Analyzing & Visualizing Data </h1>

In [52]:
# store data to csv format
df_master.to_csv('twitter_archive_master.csv', encoding='utf-8', index=False)

In [53]:
df_master = pd.read_csv('twitter_archive_master.csv')